# EDA Basic Lesson Plan

Welcome to the Exploratory Data Analysis (EDA) lesson! This notebook is designed to guide you through the essential steps of taking a raw dataset, understanding its structure, cleaning it, and transforming it for analysis.

**Structure:**
* **Section 1: The First Look:** How to read files and get a quick statistical summary.
* **Section 2: The Cleanup:** Handling the messy reality of data (missing values, duplicates, outliers).
* **Section 3: Refinement & Transformation:** preparing data for machine learning or deep analysis (mapping, string manipulation).

**For Instructors:** Use the narrative to introduce concepts before running code cells.
**For Learners:** Read the comments in the code to understand exactly what each parameter does.

In [21]:
# Typical problems with Excel data:
# 1. Inconsistent date formats
# 2. Missing values
# 3. Mixed data types in columns
# 4. Extra whitespace
# 5. Duplicates
# 6. Incorrect data types (e.g., numbers stored as text)
# 7. Merged cells
# 8. Hidden rows/columns
# 9. Special characters
# 10. Inconsistent capitalization

### Setup
As always, we start by importing the necessary libraries. `pandas` is our primary tool for data manipulation, and `numpy` helps with numerical operations.

In [22]:
import pandas as pd
import numpy as np

---

# Section 1: The First Look: Inspection & Summary

**Goal:** Load data from different file formats and get a "health check" of the data using descriptive statistics.

We will cover:
1. Reading CSV and Text files (handling headers, delimiters, and special characters).
2. Summarizing data using `.describe()`, `.sum()`, and `.value_counts()`.

## Part 1: Reading Data

Pandas is incredibly flexible with input formats. Let's start by reading a standard Comma-Separated Values (CSV) file.

In [23]:
# Let's inspect the raw file content first using a shell command
# ! marks the command to be run in the shell
!cat ../data/ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [24]:
# Use read_csv to load the data into a DataFrame
# By default, it assumes the first row is the header
df = pd.read_csv("../data/ex1.csv")

df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


**Scenario:** What if the file doesn't have a header row? If we don't specify this, pandas will mistakenly use the first row of data as column names.

In [25]:
!cat ../data/ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [26]:
# Option 1: Tell pandas there is no header. It will assign integers (0, 1, 2...) as column names.
pd.read_csv("../data/ex2.csv", header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [27]:
# Option 2: Provide your own column names using the 'names' parameter
pd.read_csv("../data/ex2.csv", names=["a", "b", "c", "d", "message"])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


**Indexing:** You can also designate a specific column to be the index (row labels) of the DataFrame, rather than the default 0, 1, 2... index.

In [28]:
names = ["a", "b", "c", "d", "message"]

# Use 'index_col' to set the 'message' column as the index
pd.read_csv("../data/ex2.csv", names=names, index_col="message")

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


**Irregular Separators:** Sometimes data isn't separated by commas. It might be tabs, spaces, or a variable amount of whitespace. We can use Regular Expressions (Regex) to handle this.

In [29]:
# Inspect a file with messy whitespace
!cat ../data/ex3.txt

A         B         C
aaa -0.264438 -1.026059 -0.619500
bbb  0.927272  0.302904 -0.032399
ccc -0.264273 -0.386314 -0.217601
ddd -0.871858 -0.348382  1.100491

In [30]:
# sep="\s+" is a regex that means "one or more whitespace characters"
result = pd.read_csv("../data/ex3.txt", sep="\s+")

# if it's a tab space, this syntax still works or alternative is sep="\t"
# result = pd.read_csv("../data/ex3.txt", sep="\t")

result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


**Skipping Rows:** Sometimes files contain comments or metadata at the top that we want to ignore.

In [31]:
!cat ../data/ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [32]:
# Skip specific rows by index (0, 2, and 3 here) to get to the clean data
pd.read_csv("../data/ex4.csv", skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


**Handling Missing Values (at load time):** Pandas is smart about identifying missing data (empty strings, 'NA', 'NULL'), but we can also define our own "sentinels" for missing values.

In [33]:
!cat ../data/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,NULL

In [34]:
# Default behavior: reads 'NA' and 'NULL' as NaN (Not a Number/Missing)
result = pd.read_csv("../data/ex5.csv")

# Pandas automatically interprets certain strings as NaN
# Sentinel - a special value to indicate missing data e.g. -9999 for temperature),
# we can specify additional strings to be treated as NaN using the na_values parameter
# sentinels = ["NULL", "NA", "---"]

result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


In [35]:
# Verify where the missing values are detected
result.isna()

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,True


You can customize what is considered "Missing" for each column specifically.

In [36]:
# Define a dictionary of sentinels
# For column 'message', treat 'foo' and 'NA' as null
# For column 'something', treat 'two' as null
sentinels = {"message": ["NULL", "NA"], "something": ["two"]}

pd.read_csv("../data/ex5.csv", na_values=sentinels, keep_default_na=False)
# keep_default_na=False prevents pandas from also treating its default strings as NaN
# define the custom missing value strings only
# na_values=sentinels means only the specified strings will be treated as NaN

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,NaN,5,6,,8,world
2,three,9,10,11,12,NaN


**Reading Excel:** Reading Excel files works similarly, but we can specify sheet names.

In [37]:
# Load the Excel file object
xlsx = pd.ExcelFile("../data/Resaleflatpricesbasedonregistrationdate.xlsx")

# List available sheets
xlsx.sheet_names

['2017', '2018']

In [38]:
# Parse a specific sheet into a DataFrame
xlsx.parse(sheet_name="2017")

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44,Improved,1979,61 years 04 months,232000
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67,New Generation,1978,60 years 07 months,250000
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67,New Generation,1980,62 years 05 months,262000
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68,New Generation,1980,62 years 01 month,265000
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67,New Generation,1980,62 years 05 months,265000
...,...,...,...,...,...,...,...,...,...,...,...
1180,2017-01,YISHUN,EXECUTIVE,387,YISHUN RING RD,07 TO 09,142,Apartment,1988,70 years 05 months,587000
1181,2017-01,YISHUN,EXECUTIVE,877,YISHUN ST 81,04 TO 06,142,Apartment,1987,69 years 11 months,590000
1182,2017-01,YISHUN,EXECUTIVE,836,YISHUN ST 81,04 TO 06,146,Maisonette,1988,70 years 01 month,673000
1183,2017-01,YISHUN,EXECUTIVE,633A,YISHUN ST 61,01 TO 03,164,Apartment,1992,74 years 05 months,678000


In [39]:
# Shortcut: Read directly without creating an ExcelFile object first
frame = pd.read_excel("../data/Resaleflatpricesbasedonregistrationdate.xlsx", sheet_name="2017")

frame

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44,Improved,1979,61 years 04 months,232000
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67,New Generation,1978,60 years 07 months,250000
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67,New Generation,1980,62 years 05 months,262000
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68,New Generation,1980,62 years 01 month,265000
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67,New Generation,1980,62 years 05 months,265000
...,...,...,...,...,...,...,...,...,...,...,...
1180,2017-01,YISHUN,EXECUTIVE,387,YISHUN RING RD,07 TO 09,142,Apartment,1988,70 years 05 months,587000
1181,2017-01,YISHUN,EXECUTIVE,877,YISHUN ST 81,04 TO 06,142,Apartment,1987,69 years 11 months,590000
1182,2017-01,YISHUN,EXECUTIVE,836,YISHUN ST 81,04 TO 06,146,Maisonette,1988,70 years 01 month,673000
1183,2017-01,YISHUN,EXECUTIVE,633A,YISHUN ST 61,01 TO 03,164,Apartment,1992,74 years 05 months,678000


## Part 2: Summarizing and Computing Descriptive Statistics

Once data is loaded, we need to understand its "shape". We use summary statistics to get a birds-eye view.

> **📊 Visual Illustration Available**: See ![descriptive_statistics](../visualisations/01_descriptive_statistics.png) for visual representations of descriptive statistics including histograms, box plots, and how statistics like mean, median, and quartiles relate to data distributions.


Let's create a sample DataFrame with some missing values to test our summary methods.

In [40]:
df = pd.DataFrame([[1.4, np.nan], [7.1, -4.5], [np.nan, np.nan], [0.75, -1.3]],
                  index=["a", "b", "c", "d"], columns=["one", "two"])

df

,one,two
a,1.40,NaN
b,7.10,-4.5
c,NaN,NaN
d,0.75,-1.3


**Reductions:** Methods like `.sum()` or `.mean()` reduce a Series of numbers to a single value. By default, they operate down the rows (`axis=0`).

In [41]:
# Sums down the rows (returns sum for each column)
df.sum()

one    9.25
two   -5.80
dtype: float64

In [42]:
# Sums across the columns (returns sum for each row)
# axis=1 is synonymous with axis='columns'
df.sum(axis="columns")

a    1.40
b    2.60
c    0.00
d   -0.55
dtype: float64

**Handling NAs in Calculations:** By default, pandas ignores NaNs (treats them as zero for sums). We can change this with `skipna`.

In [43]:
# If any value is NaN, the result is NaN
df.sum(skipna=False)

one   NaN
two   NaN
dtype: float64

In [44]:
df.sum(axis=1, skipna=False)

a     NaN
b    2.60
c     NaN
d   -0.55
dtype: float64

**Indirect Statistics:** Finding *where* the max or min value is located (the index label).

In [45]:
# Returns the index of the maximum value
df.idxmax()

one    b
two    d
dtype: object

In [46]:
# Returns the index of the minimum value
df.idxmin()

one    d
two    b
dtype: object

**Accumulations:** Computing cumulative sums.

In [47]:
df.cumsum()

,one,two
a,1.40,NaN
b,8.50,-4.5
c,NaN,NaN
d,9.25,-5.8


**The `.describe()` method:** This is your best friend for a quick snapshot of numeric data.

In [48]:
# Provides count, mean, std, min, quartiles, and max
df.describe()

,one,two
count,3.000000,2.000000
mean,3.083333,-2.900000
std,3.493685,2.262742
min,0.750000,-4.500000
25%,1.075000,-3.700000
50%,1.400000,-2.900000
75%,4.250000,-2.100000
max,7.100000,-1.300000


**Categorical/Non-Numeric Data:** `.describe()` behaves differently for string data, showing counts and uniqueness.

In [49]:
obj = pd.Series(["c", "a", "d", "a", "b", "b", "c", "c"])

# For object data, we get count, unique, top (most frequent), and freq
obj.describe()

count     8
unique    4
top       c
freq      3
dtype: object

In [50]:
# Get unique values
obj.unique()

array(['c', 'a', 'd', 'b'], dtype=object)

In [51]:
# Get frequency counts of each unique value
obj.value_counts()

c    3
a    2
b    2
d    1
dtype: int64

**Exercise:** Sort value counts in ascending order.

In [52]:
obj.value_counts().sort_index()

a    2
b    2
c    3
d    1
dtype: int64

In [53]:
obj.value_counts(ascending=True)

d    1
a    2
b    2
c    3
dtype: int64

---

# Section 2: The Cleanup: Missing Data, Duplicates & Outliers

**Goal:** Real-world data is dirty. In this section, we will learn techniques to identify and fix common quality issues.

We will cover:
1. **Missing Data:** Detecting nulls and deciding whether to drop them or fill them.
2. **Duplicates:** Finding and removing repeated records.
3. **Outliers:** identifying extreme values using boolean indexing.

## Part 1: Handling Missing Data

Missing data is often represented as `NaN` (Not a Number) or `None`.

> **📊 Visual Illustration Available**: See ![missing_data](../visualisations/03_missing_data.png) for visualizations of missing data patterns, heatmaps, and comparison of different strategies for handling missing values.


In [54]:
float_data = pd.Series([1.2, -3.5, np.nan, 0])

float_data

0    1.2
1   -3.5
2    NaN
3    0.0
dtype: float64

In [55]:
# .isna() returns a boolean mask (True if missing, False if present)
float_data.isna()

0    False
1    False
2     True
3    False
dtype: bool

The built-in Python `None` value is also treated as NA in pandas object arrays.

In [56]:
string_data = pd.Series(["aardvark", np.nan, None, "avocado"])

string_data

0    aardvark
1         NaN
2        None
3     avocado
dtype: object

In [57]:
string_data.isna()

0    False
1     True
2     True
3    False
dtype: bool

### Strategy 1: Dropping Missing Data (`dropna`)
The simplest strategy is to just remove the rows or columns that contain missing values.

In [58]:
data = pd.Series([1, np.nan, 3.5, np.nan, 7])

# Removes all NaN values from the Series
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [59]:
# This is equivalent to boolean filtering:
data[data.notna()]

0    1.0
2    3.5
4    7.0
dtype: float64

With DataFrames, `dropna` by default drops **any row** containing **any missing value**.

In [60]:
data = pd.DataFrame([[1., 6.5, 3.], [1., np.nan, np.nan], 
                     [np.nan, np.nan, np.nan], [np.nan, 6.5, 3.]])

data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [61]:
# Drops rows 1, 2, and 3 because they have at least one NaN
data.dropna()

,0,1,2
0,1.0,6.5,3.0


We can control this behavior. `how='all'` only drops rows where **all** values are NaN.

In [62]:
data.dropna(how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


To drop **columns** instead of rows, pass `axis=1`.

In [63]:
# Let's add a column of all NaNs first
data[4] = np.nan

data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [64]:
# Drops the column '4' because it is all NaNs
data.dropna(axis="columns", how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


We can also set a **threshold**: keep only rows containing at least `n` valid observations.

In [65]:
df = pd.DataFrame(np.random.standard_normal((7, 3)))
# Set some missing values
df.iloc[:4, 1] = np.nan
#:4 means from row 0 to row 3, 1 means column 1
df.iloc[:2, 2] = np.nan
#:2 means from row 0 to row 1, 2 means column 2

df

,0,1,2
0,-1.201133,NaN,NaN
1,-0.399561,NaN,NaN
2,-1.637374,NaN,1.092627
3,0.480549,NaN,-0.730390
4,0.465537,0.212938,-0.290868
5,-0.492154,-2.157631,0.298821
6,1.426323,-1.494193,-0.153106


In [66]:
df.dropna()

,0,1,2
4,0.465537,0.212938,-0.290868
5,-0.492154,-2.157631,0.298821
6,1.426323,-1.494193,-0.153106


In [67]:
# Keep rows that have at least 2 non-NaN values
df.dropna(thresh=2)
# thresh=2 means at least 2 non-NaN values are required to keep the row

,0,1,2
2,-1.637374,NaN,1.092627
3,0.480549,NaN,-0.730390
4,0.465537,0.212938,-0.290868
5,-0.492154,-2.157631,0.298821
6,1.426323,-1.494193,-0.153106


### Strategy 2: Filling Missing Data (`fillna`)
Instead of losing data, we can fill the holes with a constant or a calculated value.

In [68]:
# Replace all NaNs with 0
df.fillna(0)

,0,1,2
0,-1.201133,0.000000,0.000000
1,-0.399561,0.000000,0.000000
2,-1.637374,0.000000,1.092627
3,0.480549,0.000000,-0.730390
4,0.465537,0.212938,-0.290868
5,-0.492154,-2.157631,0.298821
6,1.426323,-1.494193,-0.153106


You can specify different fill values for each column:

In [69]:
# Fill column 1 with 0.5, and column 2 with 0
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,-1.201133,0.500000,0.000000
1,-0.399561,0.500000,0.000000
2,-1.637374,0.500000,1.092627
3,0.480549,0.500000,-0.730390
4,0.465537,0.212938,-0.290868
5,-0.492154,-2.157631,0.298821
6,1.426323,-1.494193,-0.153106


**Forward/Backward Fill:** Useful for time-series data, where you propagate the last valid observation forward or backward.

In [70]:
# Propagate next valid value backward to fill gaps
df.fillna(method="bfill")


# Time series data is a sequence of data points indexed in time order.
# This would not be useful for non-time series data.

,0,1,2
0,-1.201133,0.212938,1.092627
1,-0.399561,0.212938,1.092627
2,-1.637374,0.212938,1.092627
3,0.480549,0.212938,-0.730390
4,0.465537,0.212938,-0.290868
5,-0.492154,-2.157631,0.298821
6,1.426323,-1.494193,-0.153106


In [71]:
# Same, but limit how many rows are filled consecutively
df.fillna(method="bfill", limit=2)

,0,1,2
0,-1.201133,NaN,1.092627
1,-0.399561,NaN,1.092627
2,-1.637374,0.212938,1.092627
3,0.480549,0.212938,-0.730390
4,0.465537,0.212938,-0.290868
5,-0.492154,-2.157631,0.298821
6,1.426323,-1.494193,-0.153106


In [72]:
# This has no effect on the original DataFrame as there's no data to fill forward.
df.fillna(method="ffill")

,0,1,2
0,-1.201133,NaN,NaN
1,-0.399561,NaN,NaN
2,-1.637374,NaN,1.092627
3,0.480549,NaN,-0.730390
4,0.465537,0.212938,-0.290868
5,-0.492154,-2.157631,0.298821
6,1.426323,-1.494193,-0.153106


**Imputation:** Filling with the mean or median is a very common technique.

In [73]:
data = pd.Series([1., np.nan, 3.5, np.nan, 7])

data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [74]:
# Fill with the mean of the available data
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

### Exercise: Handling Missing Data
Practice using these methods on the dataframe below.

In [75]:
# np.random.standard_normal((6, 3)) generates a 6x3 array of random numbers from a standard normal distribution
# convert to 3 decimal places
df = pd.DataFrame(np.random.standard_normal((6, 3))).round(3)
df.iloc[[2,4,5], 1] = np.nan
df.iloc[4:, 2] = np.nan
df.iloc[3:5, 0] = np.nan

df

,0,1,2
0,-0.161,0.858,-0.621
1,1.469,-1.020,-0.003
2,-1.226,NaN,-1.027
3,NaN,0.157,0.525
4,NaN,NaN,NaN
5,-0.671,NaN,NaN


> **Tasks:**
> 1. Remove rows with *any* missing values.
> 2. Remove rows with *all* missing values.
> 3. Fill missing values with forward fill.
> 4. Fill missing values with mean of the column.

In [76]:
# remove rows with any missing values from df
df.dropna()

,0,1,2
0,-0.161,0.858,-0.621
1,1.469,-1.020,-0.003


In [77]:
# remove rows with all missing values from df
df.dropna(how="all")

,0,1,2
0,-0.161,0.858,-0.621
1,1.469,-1.020,-0.003
2,-1.226,NaN,-1.027
3,NaN,0.157,0.525
5,-0.671,NaN,NaN


In [78]:
# forward fill missing values
df.fillna(method="ffill")   

,0,1,2
0,-0.161,0.858,-0.621
1,1.469,-1.020,-0.003
2,-1.226,-1.020,-1.027
3,-1.226,0.157,0.525
4,-1.226,0.157,0.525
5,-0.671,0.157,0.525


In [79]:
# fill missing values with means of each column
df.fillna(df.mean()).round(3)

,0,1,2
0,-0.161,0.858,-0.621
1,1.469,-1.020,-0.003
2,-1.226,-0.002,-1.027
3,-0.147,0.157,0.525
4,-0.147,-0.002,-0.282
5,-0.671,-0.002,-0.282


## Part 2: Handling Duplicates

Duplicate rows can skew analysis and models. We typically identify them and remove them.

> **📊 Visual Illustration Available**: See ![duplicates](../visualisations/06_duplicates.png) for visualizations showing how duplicates affect datasets and the impact of different `keep` parameter options.


In [80]:
data = pd.DataFrame({"k1": ["one", "two"] * 3 + ["two"],
                     "k2": [1, 1, 2, 3, 3, 4, 4]})

data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


`duplicated()` returns a boolean Series indicating whether each row has been seen before.

In [81]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

`drop_duplicates()` creates a new DataFrame with the duplicates removed.

In [82]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


**Subset:** Sometimes we only care about duplicates in specific columns.

In [83]:
data["v1"] = range(7)

data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [84]:
# Drop duplicates considering only column 'k1'
data.drop_duplicates(subset=["k1"])

,k1,k2,v1
0,one,1,0
1,two,1,1


**Keep:** By default, it keeps the first occurrence. We can keep the last one instead.

In [85]:
data.drop_duplicates(subset=["k1", "k2"], keep="last")

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


## Part 3: Handling Outliers

Outliers are extreme values that deviate significantly from the rest of the data. We can filter them using boolean indexing.

> **📊 Visual Illustration Available**: See ![outliers](../visualisations/04_outliers.png) for visualizations of outlier detection methods (IQR, Z-score) and comparison of different handling strategies.


In [86]:
# Create a dataset with normal distribution
data = pd.DataFrame(np.random.standard_normal((1000, 4))).round(6)

data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.023632,-0.023891,-0.015150,-0.040410
std,1.011387,1.000643,1.017472,0.988153
min,-3.687006,-3.248243,-2.951083,-2.903471
25%,-0.597362,-0.692701,-0.721530,-0.702078
50%,0.007197,-0.059283,-0.002484,-0.060945
75%,0.664086,0.672222,0.673684,0.636752
max,3.634895,3.896595,3.140948,3.495970


**Detection:** Let's find values exceeding 3 in absolute value (Standard Deviation > 3).

In [87]:
col = data[2]

# Boolean indexing to find rows where absolute value > 3
col[col.abs() > 3]

594    3.080491
670    3.140948
Name: 2, dtype: float64

To find **any row** that has an outlier in **any column**, we use `.any(axis=1)`.

In [88]:
# 1. data.abs() > 3 returns a boolean DataFrame
# 2. .any(axis="columns") checks if any value in the row is True
data[(data.abs() > 3).any(axis="columns")]

,0,1,2,3
23,-0.109687,0.591167,-0.352352,3.495970
31,1.008049,-3.167393,0.612301,-1.495745
69,-3.687006,-0.042124,0.552878,-0.178872
101,3.358734,0.227355,0.613030,0.429555
125,3.407287,-0.135972,0.376691,-0.044473
132,-0.277261,-3.143341,0.134143,-0.541699
144,-0.747795,3.767424,1.193150,1.337733
285,-0.770332,-3.248243,0.583399,0.118619
312,0.314568,3.173053,-0.161475,0.891844
408,1.128942,3.014404,-0.896185,0.421158


**Capping:** Instead of removing outliers, we can cap them at a threshold.

In [89]:
# Set values > 3 to 3, and < -3 to -3, preserving the sign
data[data.abs() > 3] = np.sign(data) * 3

In [90]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.023212,-0.025151,-0.015372,-0.040906
std,1.003674,0.992384,1.016802,0.986500
min,-3.000000,-3.000000,-2.951083,-2.903471
25%,-0.597362,-0.692701,-0.721530,-0.702078
50%,0.007197,-0.059283,-0.002484,-0.060945
75%,0.664086,0.672222,0.673684,0.636752
max,3.000000,3.000000,3.000000,3.000000


**Removal:** Or we can just drop the rows with outliers.

In [91]:
# Keep rows where ALL columns are within the threshold ( < 3)
data[(data.abs() < 3).all(axis="columns")]

,0,1,2,3
0,-0.600993,-1.186097,0.627598,-0.250748
1,0.269340,0.171073,-0.669893,-1.380405
2,-2.200083,1.092596,-0.132803,-1.951019
3,-0.749033,-1.869592,-0.011316,1.496463
4,0.909671,-0.286563,-0.338985,0.619676
...,...,...,...,...
995,-0.265780,0.229825,-0.843602,0.013721
996,0.104856,-2.028340,0.001945,1.670935
997,1.115564,-0.250782,-1.570867,1.214188
998,0.471677,-0.042824,-1.901446,-0.182437


---

# Section 3: Refinement & Transformation

**Goal:** Now that the data is clean, we need to transform it into the right format for analysis.

We will cover:
1.  **Mapping & Replacing:** Changing values based on a dictionary logic.
2.  **String Manipulation:** Cleaning text data using the `.str` accessor.
3.  **Exporting:** Saving your hard work to a file.

## Part 1: Transforming Data (Mapping)

Sometimes we need to add new columns based on existing ones. `map()` is perfect for this.

In [92]:
data = pd.DataFrame({"food": ["bacon", "pulled pork", "bacon", "pastrami", 
                                "corned beef", "bacon", "pastrami", "honey ham", 
                                "nova lox"],
                         "ounces": [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})

data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,pastrami,6.0
4,corned beef,7.5
5,bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


**Scenario:** We want to add a column indicating the animal source of each food.

In [93]:
meat_to_animal = {
    "bacon": "pig",
    "pulled pork": "pig",
    "pastrami": "cow",
    "corned beef": "cow",
    "honey ham": "pig",
    "nova lox": "salmon"
}

In [94]:
# .map() looks up the value in the 'food' column in our dictionary
data["animal"] = data["food"].map(meat_to_animal)

data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,pastrami,6.0,cow
4,corned beef,7.5,cow
5,bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


You can also pass a function to `map()` for custom logic.

In [95]:
def get_animal(x):
    return meat_to_animal[x]

data["food"].map(get_animal)

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

**Replacing Values:** `replace` is a specialized version of map, great for fixing sentinel values (like -999 for missing data).

In [96]:
data = pd.Series([1, -999, 2, -999, -1000, 3])

data

0       1
1    -999
2       2
3    -999
4   -1000
5       3
dtype: int64

In [97]:
# Replace -999 with NaN
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [98]:
# Replace multiple values at once
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [99]:
# Replace with different values (-999 -> NaN, -1000 -> 0)
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [100]:
# Using a dictionary for clarity
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### Exercise: Transforming Data

Try replacing values in a DataFrame.

In [101]:
df = pd.DataFrame(np.random.standard_normal((6, 3)))

df.iloc[2:, 1] = -999
df.iloc[4:, 2] = 999

df

,0,1,2
0,0.463936,0.778969,0.288721
1,0.052282,-0.284044,0.772110
2,-1.043245,-999.000000,0.628299
3,-0.805663,-999.000000,-1.927847
4,-0.882275,-999.000000,999.000000
5,1.469096,-999.000000,999.000000


> **Task:** Replace -999 with NaN and 999 with 0.

In [102]:
# replace -999 with NaN and 999 with 0
df.replace({-999: "hahaha", 999: 0})

,0,1,2
0,0.463936,0.778969,0.288721
1,0.052282,-0.284044,0.772110
2,-1.043245,hahaha,0.628299
3,-0.805663,hahaha,-1.927847
4,-0.882275,hahaha,0.000000
5,1.469096,hahaha,0.000000


## Part 2: String Manipulation

Pandas has a special accessor `.str` that unlocks string methods for an entire Series at once. This handles missing values gracefully.

In [103]:
data = {"Dave": "dave@google.com", "Steve": "steve@gmail.com",
        "Rob": "rob@gmail.com", "Wes": np.nan}

data = pd.Series(data)

data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [104]:
# Check if 'gmail' exists in each string
data.str.contains("gmail")

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

Note on Data Types: Pandas has a specialized `StringDType` (`string`) vs the generic `object` type.

In [105]:
data_as_string = data.astype('string')

data_as_string
# dtype is now string

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                 <NA>
dtype: string

In [106]:
data_as_string.str.contains("gmail")

Dave     False
Steve     True
Rob       True
Wes       <NA>
dtype: boolean

**Slicing:** We can treat the column like a Python string.

In [107]:
# Get the first 5 characters
data_as_string.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes       <NA>
dtype: string

**Regex:** Regular expressions allow for complex pattern matching.

In [108]:
import re

In [109]:
# Pattern to identify email parts: (username) @ (domain) . (suffix)
pattern = r"([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})"

data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

**Retrieving elements:** We can chain `.str` calls to get specific parts of the regex match.

In [110]:
matches = data.str.findall(pattern, flags=re.IGNORECASE).str[0]

matches

Dave     (dave, google, com)
Steve    (steve, gmail, com)
Rob        (rob, gmail, com)
Wes                      NaN
dtype: object

In [111]:
# Get index 1 of the tuple (the domain name)
matches.str.get(1)

Dave     google
Steve     gmail
Rob       gmail
Wes         NaN
dtype: object

The `extract` method is very powerful—it creates a new DataFrame with columns for each captured regex group.

In [112]:
data.str.extract(pattern, flags=re.IGNORECASE)

,0,1,2
Dave,dave,google,com
Steve,steve,gmail,com
Rob,rob,gmail,com
Wes,NaN,NaN,NaN


### Exercise: Strings

> 1. Get the 1st group of the regex email pattern (the username).
> 2. Convert data into titlecase (e.g. "Dave@Google.Com").

In [113]:
# display only the username part
data.str.extract(pattern, flags=re.IGNORECASE)
data.str.extract(pattern, flags=re.IGNORECASE)[0]


Dave      dave
Steve    steve
Rob        rob
Wes        NaN
Name: 0, dtype: object

In [114]:
#convert data into title case
data.str.title()

Dave     Dave@Google.Com
Steve    Steve@Gmail.Com
Rob        Rob@Gmail.Com
Wes                  NaN
dtype: object

## Part 3: Writing Data (Exporting)

Once your data is cleaned, you need to save it. `to_csv` is the most common method.

In [115]:
result.to_csv("../data/out.csv")

In [116]:
!cat ../data/out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,


**Tip:** You usually want `index=False` to avoid saving the row numbers as a separate column.

In [117]:
result.to_csv("../data/out.csv", index=False)

In [118]:
!cat ../data/out.csv

something,a,b,c,d,message
one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,


**Excel Export:**

In [119]:
# Option 1: Using ExcelWriter (good for multiple sheets)
writer = pd.ExcelWriter('../data/out.xlsx')

frame.to_excel(writer, 'Sheet1')

writer.close()

In [122]:
# Option 2: Direct export
frame.to_excel('../data/out1.xlsx')

In [123]:
# read back the Excel file to verify
pd.read_excel('../data/out1.xlsx', 'Sheet1')

,Unnamed: 0,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44,Improved,1979,61 years 04 months,232000
1,1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67,New Generation,1978,60 years 07 months,250000
2,2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67,New Generation,1980,62 years 05 months,262000
3,3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68,New Generation,1980,62 years 01 month,265000
4,4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67,New Generation,1980,62 years 05 months,265000
...,...,...,...,...,...,...,...,...,...,...,...,...
1180,1180,2017-01,YISHUN,EXECUTIVE,387,YISHUN RING RD,07 TO 09,142,Apartment,1988,70 years 05 months,587000
1181,1181,2017-01,YISHUN,EXECUTIVE,877,YISHUN ST 81,04 TO 06,142,Apartment,1987,69 years 11 months,590000
1182,1182,2017-01,YISHUN,EXECUTIVE,836,YISHUN ST 81,04 TO 06,146,Maisonette,1988,70 years 01 month,673000
1183,1183,2017-01,YISHUN,EXECUTIVE,633A,YISHUN ST 61,01 TO 03,164,Apartment,1992,74 years 05 months,678000


---

## Optional Topics

If time permits, explore these advanced transformations.

### Renaming Axis Indexes
Changing row/column labels using mapping.

In [ ]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])

In [ ]:
data

Using `.map()` on the index:

In [ ]:
def transform(x):
    return x[:4].upper()

data.index.map(transform)

In [ ]:
# Assign back to modify in-place
data.index = data.index.map(transform)

In [ ]:
data

Using `.rename()` (returns a copy by default):

In [ ]:
data.rename(index=str.title, columns=str.upper)

In [ ]:
data.rename(index={"OHIO": "INDIANA"}, columns={"three": 3})

### Permutation and Random Sampling
Reordering or selecting random subsets.

In [ ]:
df = pd.DataFrame(np.arange(5 * 7).reshape((5, 7)))

df

In [ ]:
# Create a random order of indices
sampler = np.random.permutation(5)

sampler

In [ ]:
# Reorder rows
df.iloc[sampler]

In [ ]:
# Alternative: use .take()
df.take(sampler)

Permuting columns:

In [ ]:
column_sampler = np.random.permutation(df.shape[1])

column_sampler

In [ ]:
df.take(column_sampler, axis=1)

**Random Sample:** Getting a random subset without manually creating a sampler.

In [ ]:
df.sample(n=3)

In [ ]:
choices = pd.Series([5, 7, -1, 6, 4])

# Sample with replacement (items can be picked more than once)
choices.sample(n=10, replace=True)

> **Task:** Sample `df` using the parameter `frac` (percentage) instead of `n` (count).

### Categorical Data
Converting string columns to `category` dtype saves memory and speeds up operations.

> **📊 Visual Illustration Available**: See ![categorical_binning](../visualisations/05_categorical_binning.png` for visualizations comparing `pd.cut()` vs `pd.qcut()`, categorical operations, and dummy variables.


In [ ]:
values = pd.Series(["apple", "orange", "apple", "apple"] * 2)

values

In [ ]:
values.unique()

In [ ]:
values.value_counts()

**Under the hood:** Categoricals are stored as integers referencing a dictionary of values.

In [ ]:
values = pd.Series([0, 1, 0, 0] * 2)

dim = pd.Series(["apple", "orange"])

values

In [ ]:
dim

In [ ]:
dim.take(values)

**Using Pandas `category` type:**

In [ ]:
fruits = ["apple", "orange", "apple", "apple"] * 2
N = len(fruits)

# to ensure reproducibility
rng = np.random.default_rng(seed=12345)

df = pd.DataFrame({"fruit": fruits, 
                   "basket_id": np.arange(N), 
                   "count": rng.integers(3, 15, size=N), 
                   "weight": rng.uniform(0, 4, size=N)}, 
                  columns=["basket_id", "fruit", "count", "weight"])

df

In [ ]:
# Convert object column to category
fruit_cat = df['fruit'].astype('category')

fruit_cat

The values for `fruit_cat` are now an instance of `pandas.Categorical`, which you can access via the `.array` attribute:

In [ ]:
c = fruit_cat.array

type(c)

In [ ]:
c.categories

In [ ]:
c.codes

In [ ]:
dict(enumerate(c.categories))

In [ ]:
# Assign back to the dataframe
df['fruit'] = df['fruit'].astype('category')

df

**Creating from Codes:**

In [ ]:
categories = ['foo', 'bar', 'baz']
codes = [0, 1, 2, 0, 0, 1]

my_cats = pd.Categorical.from_codes(codes, categories)

my_cats

**Ordered Categoricals:** Useful for Likert scales or sizes (Small < Medium < Large).

In [ ]:
ordered_cats = pd.Categorical.from_codes(codes, categories, ordered=True)

ordered_cats

**Binning Data (`pd.cut`):** Converting continuous data (e.g., Age) into categorical bins (e.g., Age Groups).

In [ ]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [ ]:
# Define bin edges
bins = [18, 25, 35, 60, 100]

# Cut the data
age_cat = pd.cut(ages, bins)

age_cat

In [ ]:
age_cat.codes

In [ ]:
age_cat.categories

In [ ]:
age_cat.categories[0]

In [ ]:
pd.value_counts(age_cat)

**Binning Parameters:**

In [ ]:
# right=False makes the left side closed [inclusive, exclusive)
pd.cut(ages, bins, right=False)

In [ ]:
# Adding custom labels
group_names = ["Youth", "YoungAdult", "MiddleAged", "Senior"]

pd.cut(ages, bins, labels=group_names)

If you pass an integer number of bins to `pd.cut`, it will compute equal-length bins based on the min/max of the data.

In [ ]:
data = np.random.uniform(size=20)

pd.cut(data, 4, precision=2)

> **Task:** Cut `ages` into 5 bins instead of 4. Set the labels to `['Youth', 'YoungAdult', 'MiddleAged', 'Senior', 'Elderly']`.

**Categorical Methods (.cat):**

In [ ]:
s = pd.Series(['a', 'b', 'c', 'd'] * 2)
cat_s = s.astype('category')

cat_s

In [ ]:
cat_s.cat.codes

In [ ]:
cat_s.cat.categories

Modifying categories:

In [ ]:
actual_categories = ['a', 'b', 'c', 'd', 'e']
cat_s2 = cat_s.cat.set_categories(actual_categories)

cat_s2

In [ ]:
cat_s.value_counts()

In [ ]:
cat_s2.value_counts()

In [ ]:
cat_s2.cat.remove_unused_categories()

**Computing Indicator / Dummy Variables:** Converting categorical variables into binary columns (One-Hot Encoding) for machine learning.

In [ ]:
df = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"], 
                   "data1": range(6)})

df

In [ ]:
pd.get_dummies(df["key"])

In [ ]:
dummies = pd.get_dummies(df["key"], prefix="key")

dummies

Recipe: Combining `get_dummies` with `cut`.

In [ ]:
np.random.seed(12345)

values = np.random.uniform(size=10)

values

In [ ]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]

pd.get_dummies(pd.cut(values, bins))

> **Task:** Cut values into 4 bins and create dummy variables.

### Databases

Connecting to a database using `sqlalchemy`.

In [ ]:
import sqlalchemy as sqla

In [ ]:
import os 

parent_dir = os.path.abspath(os.path.pardir)

In [ ]:
# Create connection string
engine = sqla.create_engine(f'duckdb:///{parent_dir}/data/unit-1-4.db')

In [ ]:
# Read entire table
df = pd.read_sql('resale_flat_prices_2017', engine)

In [ ]:
df

In [ ]:
# Execute raw SQL query
df = pd.read_sql("SELECT * FROM resale_flat_prices_2017", engine)

In [ ]:
engine.table_names()

### Binary files (Pickle)

`pickle` is Python's native serialization format. Good for short-term storage, but not for sharing data between different languages.

In [ ]:
result.to_pickle('../data/out.pkl')

In [ ]:
data = pd.read_pickle('../data/out.pkl')

data

> **Task:** Write a filtered DataFrame to a new database table.
> 1. Filter for flats in "YISHUN".
> 2. Write to table `yishun_flat_prices_2017`.

In [ ]:
df_yishun = df[df.town == "YISHUN"]

df_yishun.to_sql("yishun_flat_prices_2017", engine)

> **Final Challenge:** 
> 1. Read only flats from `BISHAN` to a new dataframe.
> 2. Write the dataframe to a new database table `bishan_flat_prices_2017`.